# 参数初始化
Parameter initialization has a large impact on the model. Different initialization methods may lead to very different results. Fortunately, many pioneers of deep learning have helped us explore various initialization methods, so we only need to learn how to The assignment of the parameters of the model can be initialized.


PyTorch's initialization method is not so obvious. If you create the model in the most primitive way, then you need to define all the parameters in the model. Of course, you can easily define how each variable is initialized, but for complex models, This is not easy, and we recommend using Sequential and Module to define the model, so this time we need to know how to customize the initialization method.


## Initialize with NumPy
Because PyTorch is a very flexible framework that can theoretically operate on all Tensors, so we can initialize it by defining a new Tensor, see the example below.


In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# Define a Sequential model
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# access the parameters of the first layer
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
 0.1236 -0.1731 -0.0479  ...   0.0031  0.0784  0.1239
 0.0713  0.1615  0.0500  ...  -0.1757 -0.1274 -0.1625
 0.0638 -0.1543 -0.0362  ...   0.0316 -0.1774 -0.1242
          ...             ⋱             ...          
 0.1551  0.1772  0.1537  ...   0.0730  0.0950  0.0627
 0.0495  0.0896  0.0243  ...  -0.1302 -0.0256 -0.0326
-0.1193 -0.0989 -0.1795  ...   0.0939  0.0774 -0.0751
[torch.FloatTensor of size 40x30]



Note that this is a Parameter, which is a special Variable. We can access its `.data` property to get the data, and then directly define a new Tensor to replace it. We can use some random data in PyTorch to generate it. Ways like `torch.randn`, if you want to use more randomization methods than PyTorch, you can use numpy


In [5]:
# Define a Tensor to replace it directly
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [6]:
print(net1[0].weight)

Parameter containing:
 4.5768  3.6175  3.3098  ...   4.7374  4.0164  3.3037
 4.1809  3.5624  3.1452  ...   3.0305  4.4444  4.1058
 3.5277  4.3712  3.7859  ...   3.5760  4.8559  4.3252
          ...             ⋱             ...          
 4.8983  3.9855  3.2842  ...   4.7683  4.7590  3.3498
 4.9168  4.5723  3.5870  ...   3.2032  3.9842  3.2484
 4.2532  4.6352  4.4857  ...   3.7543  3.9885  4.4211
[torch.DoubleTensor of size 40x30]



You can see that the value of this parameter has been changed, that is, it has been defined as the initialization method we need. If a layer in the model needs us to modify it manually, then we can directly access it in this way, but More often than not, the same type of layer in the model needs to be initialized in the same way. A more efficient way to do this is to use a loop to access it.


In [7]:
for layer in net1:
If isinstance(layer, nn.Linear): # Determine if it is a linear layer
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
# Defined as a normal distribution with a mean of 0 and a variance of 0.5


**Small practice: A very popular initialization method called Xavier, the method comes from a 2010 paper [Understanding the difficulty of training deep feedforward neural networks] (http:

We give this formula for initialization

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

Where $n_j$ and $n_{j 1}$ represent the number of inputs and outputs for this layer, so try to implement the following initialization method


For the parameter initialization of the Module, it is also very simple. If you want to initialize one of the layers, you can redefine the Tensor directly like Sequential. The only difference is that if you want to access it in a loop, you need to introduce it. Two properties, children and modules, let's take an example


In [8]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
Self.l1[0].weight.data = torch.randn(40, 30) # Initialize a layer directly
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [9]:
net2 = sim_net()

In [10]:
#访 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)


In [11]:
# access modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Linear(in_features=30, out_features=40)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Linear(in_features=40, out_features=50)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)
Linear(in_features=50, out_features=10)
ReLU()


See the difference through the above example?

Children will only access the first layer in the model definition, because the above model defines three Sequential, so only three Sequential are accessed, and modules will access the last structure. For example, the above example, modules not only access When I arrived at Sequential, I also visited Sequential, which is very convenient for us to initialize.


In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

This achieves the same initialization as Sequential, which is also very easy


## torch.nn.init
Because of the flexible nature of PyTorch, we can directly operate on Tensor for initialization. PyTorch also provides initialization functions to help us quickly initialize, which is `torch.nn.init`. The operation level is still on Tensor. Below we give an example.


In [13]:
from torch.nn import init

In [14]:
print(net1[0].weight)

Parameter containing:
 0.8453  0.2891 -0.5276  ...  -0.1530 -0.4474 -0.5470
-0.1983 -0.4530 -0.1950  ...   0.4107 -0.4889  0.3654
 0.9149 -0.5641 -0.6594  ...   0.0734  0.1354 -0.4152
          ...             ⋱             ...          
-0.4718 -0.5125 -0.5572  ...   0.0824 -0.6551  0.0840
-0.2374 -0.0036  0.6497  ...   0.7856 -0.1367 -0.8795
 0.0774  0.2609 -0.2358  ...  -0.8196  0.1696  0.5976
[torch.DoubleTensor of size 40x30]



In [15]:
Init.xavier_uniform(net1[0].weight) # This is the Xavier initialization method we talked about above. PyTorch has its built-in implementation directly.


Parameter containing:
-0.2114  0.2704 -0.2186  ...   0.1727  0.2158  0.0775
-0.0736 -0.0565  0.0844  ...   0.1793  0.2520 -0.0047
 0.1331 -0.1843  0.2426  ...  -0.2199 -0.0689  0.1756
          ...             ⋱             ...          
 0.2751 -0.1404  0.1225  ...   0.1926  0.0175 -0.2099
 0.0970 -0.0733 -0.2461  ...   0.0605  0.1915 -0.1220
 0.0199  0.1283 -0.1384  ...  -0.0344 -0.0560  0.2285
[torch.DoubleTensor of size 40x30]

In [16]:
print(net1[0].weight)

Parameter containing:
-0.2114  0.2704 -0.2186  ...   0.1727  0.2158  0.0775
-0.0736 -0.0565  0.0844  ...   0.1793  0.2520 -0.0047
 0.1331 -0.1843  0.2426  ...  -0.2199 -0.0689  0.1756
          ...             ⋱             ...          
 0.2751 -0.1404  0.1225  ...   0.1926  0.0175 -0.2099
 0.0970 -0.0733 -0.2461  ...   0.0605  0.1915 -0.1220
 0.0199  0.1283 -0.1384  ...  -0.0344 -0.0560  0.2285
[torch.DoubleTensor of size 40x30]



Can see that the parameters have been modified

`torch.nn.init` gives us more built-in initialization methods, avoiding us repeating some of the same operations


The above two methods of initialization, in fact, their essence is the same, is to modify the actual value of a layer of parameters, and `torch.nn.init` provides more mature deep learning related initialization methods, very Convenience

In the next lesson, we will talk about the various gradient-based optimization algorithms that are currently popular.
